In [ ]:
import numpy as np
from astropy.io import fits
import datetime

# -----------------------------
# Create the Primary HDU Header (MUSE-specific)
# -----------------------------
prim_hdr = fits.Header()

# Standard FITS keywords
prim_hdr['SIMPLE']   = True                          # Conforms to FITS standard
prim_hdr['BITPIX']   = -32                           # 32-bit floating-point
prim_hdr['NAXIS']    = 0                             # No data array in primary HDU
prim_hdr['EXTEND']   = True                          # File may contain extensions
prim_hdr['DATE']     = datetime.datetime.now().strftime('%Y-%m-%d')

# MUSE Phase 3 Primary Header Keywords
prim_hdr['PRODCATG'] = 'SCIENCE.SPECTRUM'            # Data product category
# prim_hdr['ASSOC1']   = 'MUSE_ASSOC_001'              # Association identifier
# prim_hdr['ASSON1']   = 'muse_associated.fits'          # Associated file name
# prim_hdr['ASSOM1']   = 'd41d8cd98f00b204e9800998ecf8427e'  # MD5 checksum (dummy)
prim_hdr['ORIGIN']   = 'ESO'
prim_hdr['TELESCOP'] = 'VLT'                           # Telescope name
prim_hdr['INSTRUME'] = 'MUSE'                          # Instrument name
prim_hdr['OBJECT']   = 'MUSE_Target'                   # Target name
prim_hdr['RA']       = 10.684                        # Right Ascension in degrees (dummy)
prim_hdr['DEC']      = 41.269                        # Declination in degrees (dummy)
prim_hdr['EQUINOX']  = 2000.0                        # Equinox
prim_hdr['RADESYS']  = 'FK5'
prim_hdr['TIMESYS']  = 'UTC'
prim_hdr['EXPTIME']  = 1800.0                        # Exposure time per bin (s)
prim_hdr['TEXPTIME'] = 1800.0                        # Total exposure time (s)
prim_hdr['MJD-OBS']  = 59300.0                       # Observation start (MJD)
prim_hdr['MJD-END']  = 59300.0208                    # Observation end (MJD; ~1800 s = 0.0208 days)
prim_hdr['PROG_ID']  = '0102.B-1234(A)'              # ESO Program ID
prim_hdr['OBID1']    = 'MUSE_OBS_001'                # Observation ID
prim_hdr['NCOMBINE'] = 1                             # Number of combined exposures
prim_hdr['OBSTECH']  = 'Integral Field Spectroscopy'
prim_hdr['FLUXCAL']  = 'ABSOLUTE'                    # Flux calibration status
prim_hdr['PROCSOFT'] = 'MUSE Pipeline v2.8'           # Processing software
prim_hdr['REFERENC'] = 'Ref. MUSE 2020'               # Bibliographic reference
prim_hdr['PROV1']    = 'MUSE Reduction Pipeline'       # Provenance info
# BUNIT and CDi_j are not allowed in the PrimaryfTEXPTIME HDU.
prim_hdr['SPECSYS']  = 'BARYCENT'                    # Spectral reference system
prim_hdr['EXT_OBJ']  = 'MUSE_Target'                 # External object identifier
prim_hdr['CONTNORM'] = True                         # Continuum normalization flag
prim_hdr['TOT_FLUX'] = 2.5e-16                      # Total flux value (dummy)
prim_hdr['FLUXERR']  = 1.0e-17                      # Global flux error (dummy)
prim_hdr['WAVELMIN'] = 4650.0                       # Minimum wavelength (Å); typical for MUSE
prim_hdr['WAVELMAX'] = 9300.0                       # Maximum wavelength (Å)
prim_hdr['LAMRMS']   = 0.0                         # RMS deviation of wavelength solution (optional)
prim_hdr['LAMNLIN']  = 0                           # Number of fitted wavelength lines (optional)
prim_hdr['SPEC_BIN'] = 1.25                        # Spectral bin width (Å)
prim_hdr['SPEC_ERR'] = 1.0e-17                      # Spectral error estimate (recommended)
prim_hdr['SPEC_SYE'] = 1.0e-18                      # Systematic spectral error (recommended)
prim_hdr['RA_ERR']   = 0.0001                       # RA uncertainty (recommended)
prim_hdr['DEC_ERR']  = 0.0001                       # DEC uncertainty (recommended)
prim_hdr['STOKES']   = 'I'                         # Stokes parameter (Intensity)
prim_hdr['SNR']      = 60                          # Signal-to-noise ratio (dummy)
prim_hdr['SPEC_RES'] = 3000                        # Spectral resolution (approximate for MUSE)
prim_hdr['STREHL']   = 0.8                         # Strehl ratio (AO; dummy)
prim_hdr['ARCFILE']  = ''                          # Archive file name (reserved)
prim_hdr['CHECKSUM'] = ''                          # Checksum (to be updated)
prim_hdr['DATASUM']  = ''                          # Datasum (to be updated)
prim_hdr['ORIGFILE'] = ''                          # Original file name (reserved)
prim_hdr['P3ORIG']   = ''                          # Original Phase 3 identifier (reserved)
prim_hdr['NOESODAT'] = 'T'                         # Non-ESO proprietary data flag

primary_hdu = fits.PrimaryHDU(header=prim_hdr)

# -----------------------------
# Create the Binary Table Extension (MUSE Extracted Spectrum)
# -----------------------------
n = 100  # Number of spectral points

# Create dummy arrays for a MUSE extracted 1D spectrum
# For MUSE, the wavelength range typically spans 4650 to 9300 Å.
wave = np.linspace(4650, 9300, n)
flux = np.ones(n) * 2.5e-16      # Dummy flux in erg/s/cm^2/Å
err  = np.ones(n) * 1.0e-17      # Dummy 1-sigma error (same units as flux)

# Each column is stored as a single cell containing a vector.
col_wave = fits.Column(name='WAVE', array=[wave], format=f'{n}E', unit='Angstrom')
col_flux = fits.Column(name='FLUX', array=[flux], format=f'{n}E',
                       unit='erg/s/cm^2/Angstrom')
col_err  = fits.Column(name='ERR',  array=[err],  format=f'{n}E',
                       unit='erg/s/cm^2/Angstrom')

cols = fits.ColDefs([col_wave, col_flux, col_err])
bintable = fits.BinTableHDU.from_columns(cols)

# Set additional Extension Header Keywords for Phase 3 and VO compliance
ext_hdr = bintable.header
ext_hdr['RA']       = 10.684                         # (repeated) RA in degrees
ext_hdr['DEC']      = 41.269                         # (repeated) DEC in degrees
ext_hdr['NELEM']    = n                              # Number of elements in each vector
ext_hdr['VOCLASS']  = 'SPECTRUM v1.0'                # VO Data Model version
ext_hdr['VOPUB']    = 'Published'                    # VO publication status
ext_hdr['TITLE']    = 'MUSE Extracted Spectrum'      # Spectrum title
ext_hdr['APERTURE'] = 1.0                            # Aperture size (arcsec; dummy)
ext_hdr['TELAPSE']  = 1800.0                         # Elapsed time in seconds
ext_hdr['TMID']     = 59300.0104                     # Midpoint of observation (MJD)
ext_hdr['SPEC_VAL'] = (4650.0 + 9300.0) / 2.0          # Central wavelength (Å)
ext_hdr['SPEC_BW']  = 9300.0 - 4650.0                # Spectral bandwidth (Å)
ext_hdr['ARCFILE']  = ''                           # Archive file name (reserved)
ext_hdr['CHECKSUM'] = ''                           # Checksum (to be updated)
ext_hdr['DATASUM']  = ''                           # Datasum (to be updated)
ext_hdr['ORIGFILE'] = ''                           # Original file name (reserved)
ext_hdr['P3ORIG']   = ''                           # Original Phase 3 identifier (reserved)
ext_hdr['TFIELDS']  = 3                            # Number of table columns

# Column definitions: TTYPEi, TFORMi, TCOMMi, TUNITi, TUTYPi, TUCDi, TDMINi, TDMAXi
ext_hdr['TTYPE1']   = 'WAVE'
ext_hdr['TTYPE2']   = 'FLUX'
ext_hdr['TTYPE3']   = 'ERR'

ext_hdr['TFORM1']   = f'{n}E'
ext_hdr['TFORM2']   = f'{n}E'
ext_hdr['TFORM3']   = f'{n}E'

ext_hdr['TCOMM1']   = 'Wavelength array'
ext_hdr['TCOMM2']   = 'Flux array'
ext_hdr['TCOMM3']   = 'Error array'

ext_hdr['TUNIT1']   = 'Angstrom'
ext_hdr['TUNIT2']   = 'erg/s/cm^2/Angstrom'
ext_hdr['TUNIT3']   = 'erg/s/cm^2/Angstrom'

ext_hdr['TUTYP1']   = 'Spectrum.Data.SpectralAxis.Value'
ext_hdr['TUTYP2']   = 'Spectrum.Data.FluxAxis.Value'
ext_hdr['TUTYP3']   = 'Spectrum.Data.FluxAxis.Accuracy.StatError'

ext_hdr['TUCD1']    = 'em.wl;obs.atmos'
ext_hdr['TUCD2']    = 'phot.flux.density;em.wl'
ext_hdr['TUCD3']    = 'stat.error;phot.flux.density;em.wl'

ext_hdr['TDMIN1']   = wave.min()
ext_hdr['TDMAX1']   = wave.max()
ext_hdr['TDMIN2']   = flux.min()
ext_hdr['TDMAX2']   = flux.max()
ext_hdr['TDMIN3']   = err.min()
ext_hdr['TDMAX3']   = err.max()

ext_hdr['EXTNAME']  = 'SCIENCE.SPECTRUM'

# -----------------------------
# Create the HDU List and Write the FITS File
# -----------------------------
hdulist = fits.HDUList([primary_hdu, bintable])

# Update checksums for HDUs that support it
for hdu in hdulist:
    if hasattr(hdu, 'update_checksum'):
        hdu.update_checksum()

hdulist.writeto('../tmp/dummy_muse_spectrum.fits', overwrite=True)
print("Dummy MUSE spectrum saved to '../tmp/dummy_muse_spectrum.fits'")

Dummy MUSE spectrum saved to '../tmp/dummy_muse_spectrum.fits'


In [2]:
from astropy.io import fits
hdu = fits.open('../tmp/dummy_spectrum.fits')
hdu.info()

Filename: ../tmp/dummy_spectrum.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      25   ()      
  1  SCIENCE.SPECTRUM    1 BinTableHDU     26   1R x 3C   [100E, 100E, 100E]   
